In [1]:
import json
import pickle
import pandas as pd
import os

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator
import torch.nn.functional as F
import spacy
import numpy as np

import random
import math
import time

from torchtext.data import TabularDataset

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
!wget https://s3.amazonaws.com/commensenseqa/train_rand_split.jsonl

--2020-12-30 07:36:48--  https://s3.amazonaws.com/commensenseqa/train_rand_split.jsonl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.154.158
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.154.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785890 (3.6M) [binary/octet-stream]
Saving to: ‘train_rand_split.jsonl’

train_rand_split.js 100%[===================>]   3.61M  --.-KB/s    in 0.09s   

2020-12-30 07:36:49 (40.2 MB/s) - ‘train_rand_split.jsonl’ saved [3785890/3785890]



In [4]:
!wget https://s3.amazonaws.com/commensenseqa/dev_rand_split.jsonl

--2020-12-30 07:36:49--  https://s3.amazonaws.com/commensenseqa/dev_rand_split.jsonl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.154.158
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.154.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 471653 (461K) [binary/octet-stream]
Saving to: ‘dev_rand_split.jsonl’

dev_rand_split.json 100%[===================>] 460.60K  --.-KB/s    in 0.02s   

2020-12-30 07:36:49 (19.1 MB/s) - ‘dev_rand_split.jsonl’ saved [471653/471653]



In [5]:
df = pd.read_json('/content/dev_rand_split.jsonl',lines= True)

In [6]:
df.head()

,answerKey,id,question
0,A,1afa02df02c908a558b4036e80242fac,"{'question_concept': 'revolving door', 'choice..."
1,A,a7ab086045575bb497933726e4e6ad28,"{'question_concept': 'people', 'choices': [{'l..."
2,B,b8c0a4703079cf661d7261a60a1bcbff,"{'question_concept': 'magazines', 'choices': [..."
3,A,e68fb2448fd74e402aae9982aa76e527,"{'question_concept': 'hamburger', 'choices': [..."
4,A,2435de612dd69f2012b9e40d6af4ce38,"{'question_concept': 'farmland', 'choices': [{..."


In [7]:
question = []
answer = []
for i in range(df.shape[0]):
  question.append(df.iloc[i].question['stem'])
  choice = df.iloc[i].question['choices']
  for ii in range(len(choice)):
    if choice[ii]['label'] == df.iloc[i].answerKey:
      answer.append(choice[ii]['text'])





In [8]:
df['Question'] = question
df['Answer'] = answer

In [9]:
df.head()

,answerKey,id,question,Question,Answer
0,A,1afa02df02c908a558b4036e80242fac,"{'question_concept': 'revolving door', 'choice...",A revolving door is convenient for two directi...,bank
1,A,a7ab086045575bb497933726e4e6ad28,"{'question_concept': 'people', 'choices': [{'l...",What do people aim to do at work?,complete job
2,B,b8c0a4703079cf661d7261a60a1bcbff,"{'question_concept': 'magazines', 'choices': [...",Where would you find magazines along side many...,bookstore
3,A,e68fb2448fd74e402aae9982aa76e527,"{'question_concept': 'hamburger', 'choices': [...",Where are you likely to find a hamburger?,fast food restaurant
4,A,2435de612dd69f2012b9e40d6af4ce38,"{'question_concept': 'farmland', 'choices': [{...",James was looking for a good place to buy farm...,midwest


In [10]:
dev_df = df[['Question','Answer']]
dev_df.head()

,Question,Answer
0,A revolving door is convenient for two directi...,bank
1,What do people aim to do at work?,complete job
2,Where would you find magazines along side many...,bookstore
3,Where are you likely to find a hamburger?,fast food restaurant
4,James was looking for a good place to buy farm...,midwest


In [11]:
if not os.path.exists('preprocessed_valid.csv'):
    dev_df.to_csv('preprocessed_valid.csv', index = False)

In [12]:
train_df =  pd.read_json('/content/train_rand_split.jsonl',lines= True) 

In [13]:
train_df.head()

,answerKey,id,question
0,A,075e483d21c29a511267ef62bedc0461,"{'question_concept': 'punishing', 'choices': [..."
1,B,61fe6e879ff18686d7552425a36344c8,"{'question_concept': 'people', 'choices': [{'l..."
2,A,4c1cb0e95b99f72d55c068ba0255c54d,"{'question_concept': 'choker', 'choices': [{'l..."
3,D,02e821a3e53cb320790950aab4489e85,"{'question_concept': 'highway', 'choices': [{'..."
4,C,23505889b94e880c3e89cff4ba119860,"{'question_concept': 'fox', 'choices': [{'labe..."


In [14]:
question = []
answer = []
for i in range(train_df.shape[0]):
  question.append(train_df.iloc[i].question['stem'])
  choice = train_df.iloc[i].question['choices']
  for ii in range(len(choice)):
    if choice[ii]['label'] == train_df.iloc[i].answerKey:
      answer.append(choice[ii]['text'])

In [15]:
train_df['Question'] = question
train_df['Answer'] = answer

In [16]:
train_df = train_df[['Question','Answer']]
train_df.head()

,Question,Answer
0,The sanctions against the school were a punish...,ignore
1,Sammy wanted to go to where the people were. ...,populated areas
2,To locate a choker not located in a jewelry bo...,jewelry store
3,Google Maps and other highway and street GPS s...,atlas
4,"The fox walked from the city into the forest, ...",natural habitat


In [17]:
if not os.path.exists('preprocessed_train.csv'):
    train_df.to_csv('preprocessed_train.csv', index = False)

In [18]:
spacy_en = spacy.load('en')

In [19]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [20]:
QSTN = Field(tokenize=tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

ANS = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [21]:
trainset = TabularDataset(path='preprocessed_train.csv', format='csv', skip_header=True,
                            fields=[('Question', QSTN), ('Answer', ANS)])

In [22]:
test_data = TabularDataset(path='preprocessed_valid.csv', format='csv', skip_header=True,
                            fields=[('Question', QSTN), ('Answer', ANS)])

In [23]:
train_data, valid_data = trainset.split(split_ratio = 0.7, random_state=random.getstate())  

In [24]:
vars(train_data.examples[0])

{'Answer': ['fifties'],
 'Question': ['jerry',
  'was',
  'too',
  'old',
  'for',
  'this',
  'shit',
  '.',
  ' ',
  'he',
  'was',
  'too',
  'square',
  ',',
  'too',
  '.',
  ' ',
  'he',
  'never',
  'did',
  'drugs',
  ',',
  'never',
  'drank',
  ',',
  'but',
  'he',
  'was',
  'in',
  'his',
  'what',
  '?']}

In [25]:
vars(valid_data.examples[0])

{'Answer': ['maternity', 'ward'],
 'Question': ['where',
  'would',
  'you',
  'find',
  'a',
  'waiting',
  'room',
  'with',
  'only',
  'some',
  'fathers',
  '?']}

In [26]:
QSTN.build_vocab(train_data, min_freq = 2)
ANS.build_vocab(train_data, min_freq = 2)

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort_key=lambda x: len(x.Question), # the BucketIterator needs to be told what function it should use to group the data.
 sort_within_batch=False,)

In [29]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [30]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [31]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [32]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [33]:
INPUT_DIM = len(QSTN.vocab)
OUTPUT_DIM = len(ANS.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [34]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(3725, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(1454, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=1454, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,366,126 trainable parameters


In [36]:
optimizer = optim.Adam(model.parameters())

In [37]:
TRG_PAD_IDX = ANS.vocab.stoi[ANS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [38]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.Question
        trg = batch.Answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [39]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.Question
            trg = batch.Answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [40]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [41]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 4s
	Train Loss: 4.585 | Train PPL:  97.965
	 Val. Loss: 3.742 |  Val. PPL:  42.165
Epoch: 02 | Time: 0m 4s
	Train Loss: 3.926 | Train PPL:  50.696
	 Val. Loss: 3.787 |  Val. PPL:  44.144
Epoch: 03 | Time: 0m 4s
	Train Loss: 3.822 | Train PPL:  45.691
	 Val. Loss: 3.671 |  Val. PPL:  39.288
Epoch: 04 | Time: 0m 4s
	Train Loss: 3.758 | Train PPL:  42.863
	 Val. Loss: 3.749 |  Val. PPL:  42.494
Epoch: 05 | Time: 0m 4s
	Train Loss: 3.678 | Train PPL:  39.571
	 Val. Loss: 3.674 |  Val. PPL:  39.423
Epoch: 06 | Time: 0m 4s
	Train Loss: 3.557 | Train PPL:  35.075
	 Val. Loss: 3.819 |  Val. PPL:  45.536
Epoch: 07 | Time: 0m 4s
	Train Loss: 3.423 | Train PPL:  30.668
	 Val. Loss: 3.676 |  Val. PPL:  39.503
Epoch: 08 | Time: 0m 4s
	Train Loss: 3.322 | Train PPL:  27.717
	 Val. Loss: 3.740 |  Val. PPL:  42.080
Epoch: 09 | Time: 0m 4s
	Train Loss: 3.200 | Train PPL:  24.536
	 Val. Loss: 3.727 |  Val. PPL:  41.548
Epoch: 10 | Time: 0m 4s
	Train Loss: 3.077 | Train PPL:  21.692


In [42]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.652 | Test PPL:  38.563 |
